In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from demand_features import rf_cols
from price_optimize import update_rates

pd.options.display.max_rows = 160
pd.options.display.max_columns = 250

df_sim = pd.read_pickle("../data/h1_stats.pick")

DATE_FMT = "%Y-%m-%d"
as_of_date = '2017-08-01'

In [2]:
def splits(df_sim, as_of_date):
    """
    Splits df_sim into X_train, X_test, y_train, y_test.
    """
    train_mask = df_sim["StayDate"] < as_of_date
    test_mask = df_sim["AsOfDate"] == as_of_date
    df_train = df_sim.loc[train_mask].copy()
    df_test = df_sim.loc[test_mask].copy()

    X_train = df_train[rf_cols].copy()
    X_test = df_test[rf_cols].copy()
    y_train = df_train["ACTUAL_TRN_RoomsPickup"].copy()
    y_test = df_test["ACTUAL_TRN_RoomsPickup"].copy()

    return X_train, y_train, X_test, y_test

In [3]:
# splits func

train_mask = df_sim["StayDate"] < as_of_date
test_mask = df_sim["AsOfDate"] == as_of_date
df_train = df_sim.loc[train_mask].copy()
df_test = df_sim.loc[test_mask].copy()

X_train = df_train[rf_cols].copy()
X_test = df_test[rf_cols].copy()
y_train = df_train["ACTUAL_TRN_RoomsPickup"].copy()
y_test = df_test["ACTUAL_TRN_RoomsPickup"].copy()

In [4]:
# train_model func
X_train, y_train, X_test, y_test = splits(df_sim, as_of_date)

rfm = RandomForestRegressor(
    n_estimators=150,
    max_depth=48,
    min_samples_split=2,
    bootstrap=True,
    n_jobs=-1,
    random_state=20,
)
rfm.fit(X_train, y_train)
preds = rfm.predict(X_test)
print("Finished training model.")
r2 = round(r2_score(y_test, preds), 4)
print(f"R² Score on test set: {r2}")

X_test["ACTUAL_TRN_RoomsPickup"] = preds.round(0).astype(int)
df_sim["ACTUAL_TRN_RoomsPickup"] = X_test["ACTUAL_TRN_RoomsPickup"]
mask = df_sim["AsOfDate"] == as_of_date
df_pricing = df_sim[mask].copy()

Finished training model.
R² Score on test set: 0.6789


In [5]:
df_pricing

,id,DOW,RoomsOTB,RevOTB,CxlForecast,TRN_RoomsOTB,TRN_RevOTB,TRN_CxlForecast,TRNP_RoomsOTB,GRP_RoomsOTB,CNT_RoomsOTB,StayDate,STLY_StayDate,DaysUntilArrival,Realized_Cxls,SellingPrice,TM30_RoomsOTB,TM30_RevOTB,TM30_TRN_RoomsOTB,TM30_TRN_RevOTB,TM15_RoomsOTB,TM15_RevOTB,TM15_TRN_RoomsOTB,TM15_TRN_RevOTB,TM05_RoomsOTB,TM05_RevOTB,TM05_TRN_RoomsOTB,TM05_TRN_RevOTB,AsOfDate,STLY_AsOfDate,RemSupply,ADR_OTB,TRN_ADR_OTB,LYA_RoomsSold,LYA_ADR,LYA_RoomRev,LYA_NumCancels,LYA_TRN_RoomsSold,LYA_TRN_ADR,LYA_TRN_RoomRev,ACTUAL_RoomsSold,ACTUAL_ADR,ACTUAL_RoomRev,ACTUAL_TRN_RoomsSold,ACTUAL_TRN_ADR,ACTUAL_TRN_RoomRev,ACTUAL_NumCancels,ACTUAL_RoomsPickup,ACTUAL_ADR_Pickup,ACTUAL_RevPickup,ACTUAL_TRN_RoomsPickup,ACTUAL_TRN_ADR_Pickup,ACTUAL_TRN_RevPickup,MonthNum,DayOfWeek,Mon,Sat,Sun,Thu,Tue,Wed,WE,week_of_year,TM30_ADR_OTB,TM30_RoomsPickup,TM30_RevPickup,TM30_ADR_Pickup,TM30_TRN_ADR_OTB,TM30_TRN_RoomsPickup,TM30_TRN_RevPickup,TM30_TRN_ADR_Pickup,TM15_ADR_OTB,TM15_RoomsPickup,TM15_RevPickup,TM15_ADR_Pickup,TM15_TRN_ADR_OTB,TM15_TRN_RoomsPickup,TM15_TRN_RevPickup,TM15_TRN_ADR_Pickup,TM05_ADR_OTB,TM05_RoomsPickup,TM05_RevPickup,TM05_ADR_Pickup,TM05_TRN_ADR_OTB,TM05_TRN_RoomsPickup,TM05_TRN_RevPickup,TM05_TRN_ADR_Pickup,OTB_GapToLYA_RoomsSold,OTB_GapToLYA_ADR,OTB_GapToLYA_RoomRev,OTB_GapToLYA_NumCancels,OTB_GapToLYA_TRN_RoomsSold,OTB_GapToLYA_TRN_ADR,OTB_GapToLYA_TRN_RoomRev,id_STLY,AsOfDate_STLY,StayDate_STLY,RoomsOTB_STLY,ADR_OTB_STLY,RevOTB_STLY,ACTUAL_RoomsPickup_STLY,ACTUAL_ADR_Pickup_STLY,ACTUAL_RevPickup_STLY,CxlForecast_STLY,RemSupply_STLY,SellingPrice_STLY,Realized_Cxls_STLY,TRN_RoomsOTB_STLY,TRN_ADR_OTB_STLY,TRN_RevOTB_STLY,TRN_CxlForecast_STLY,ACTUAL_TRN_RoomsPickup_STLY,ACTUAL_TRN_ADR_Pickup_STLY,ACTUAL_TRN_RevPickup_STLY,TM30_RoomsPickup_STLY,TM30_ADR_Pickup_STLY,TM30_RevPickup_STLY,TM30_TRN_RoomsPickup_STLY,TM30_TRN_ADR_Pickup_STLY,TM30_TRN_RevPickup_STLY,TM15_RoomsPickup_STLY,TM15_ADR_Pickup_STLY,TM15_RevPickup_STLY,TM15_TRN_RoomsPickup_STLY,TM15_TRN_ADR_Pickup_STLY,TM15_TRN_RevPickup_STLY,TM05_RoomsPickup_STLY,TM05_ADR_Pickup_STLY,TM05_RevPickup_STLY,TM05_TRN_RoomsPickup_STLY,TM05_TRN_ADR_Pickup_STLY,TM05_TRN_RevPickup_STLY,Pace_RoomsOTB,Pace_ADR_OTB,Pace_RevOTB,Pace_CxlForecast,Pace_RemSupply,Pace_SellingPrice,Pace_TRN_RoomsOTB,Pace_TRN_ADR_OTB,Pace_TRN_RevOTB,Pace_TRN_CxlForecast,Pace_TM30_RoomsPickup,Pace_TM30_ADR_Pickup,Pace_TM30_RevPickup,Pace_TM30_TRN_RoomsPickup,Pace_TM30_TRN_ADR_Pickup,Pace_TM30_TRN_RevPickup,Pace_TM15_RoomsPickup,Pace_TM15_ADR_Pickup,Pace_TM15_RevPickup,Pace_TM15_TRN_RoomsPickup,Pace_TM15_TRN_ADR_Pickup,Pace_TM15_TRN_RevPickup,Pace_TM05_RoomsPickup,Pace_TM05_ADR_Pickup,Pace_TM05_RevPickup,Pace_TM05_TRN_RoomsPickup,Pace_TM05_TRN_ADR_Pickup,Pace_TM05_TRN_RevPickup
11712,2017-08-01 - 2017-08-01,Tue,184.0,35239.14,18.0,151.0,29935.72,16.0,13.0,2.0,18.0,2017-08-01,2016-08-02,0.0,134.0,204.00,170.0,31991.18,139.0,27227.18,180.0,34519.71,147.0,29216.29,177.0,33992.74,144.0,28689.32,2017-08-01,2016-08-02,21.0,191.52,198.25,183.0,170.00,31110.89,81.0,159.0,175.03,27829.05,184.0,191.52,35239.14,151.0,198.25,29935.72,134.0,0.0,0.00,0.00,-1.0,0.00,0.00,8,Tue,False,False,0,False,True,False,False,31.0,188.18,14.0,3247.96,3.34,195.88,12.0,2708.54,2.37,191.78,4.0,719.43,-0.26,198.75,4.0,719.43,-0.50,192.05,7.0,1246.40,-0.53,199.23,7.0,1246.40,-0.98,-1.0,-21.52,-4128.25,-53.0,8.0,-23.22,-2106.67,2016-08-02 - 2016-08-02,2016-08-02,2016-08-02,183.0,170.00,31110.89,0.0,0.00,0.00,39.0,43.0,178.09,81.0,159.0,175.03,27829.05,34.0,0.0,0.00,0.00,3.0,4.00,1230.63,3.0,4.53,1230.63,3.0,2.64,985.79,3.0,2.96,985.79,1.0,1.88,512.25,1.0,2.14,512.25,1.0,21.52,4128.25,-21.0,-22.0,25.91,-8.0,23.22,2106.67,-18.0,11.0,-0.66,2017.33,9.0,-2.16,1477.91,1.0,-2.90,-266.36,1.0,-3.46,-266.36,6.0,-2.41,734.15,6.0,-3.12,734.15
11713,2017-08-01 - 2017-08-02,Wed,185.0,35961.47,19.0,151.0,30760.93,17.0,11.0,2.0,21.0,2017-08-02,2016-08-03,1.0,131.0,206.75,174.0,32687.77,143.0,28148.15,183.0,35208.06,149.0,30007.52,183.0,35456.32,149.0,30255.78,2017-08-01

In [6]:
model = rfm

In [7]:
def calculate_rev_at_price(price, df_sim, model, df_index):
    df = df_sim.copy()
    df.loc[i, "SellingPrice"] = price
    X = df.loc[i, rf_cols].to_numpy()
    trn_rooms_to_book = model.predict([X])[0]
    resulting_rev = round(trn_rooms_to_book * price, 2)
    return resulting_rev

In [8]:
# optimize_price func
indices = list(df_pricing.index)
price_adjustments = np.delete(np.arange(-0.25, 0.30, 0.05).round(2), 5)
optimal_prices = []
for i in indices:
#     sd = dt.datetime.strftime(stay_date, format=DATE_FMT)
    original_rate = round(df_pricing.loc[i, "SellingPrice"], 2)
    date_X = df_pricing.loc[i, rf_cols].to_numpy()
    pred = model.predict([date_X])[0]
    original_rev = pred * original_rate
    optimal_rate = (original_rate, original_rev, original_rate, original_rev)

    for pct in price_adjustments:
        new_rate = round(original_rate * (1 + pct), 2)
        resulting_rev = calculate_rev_at_price(new_rate, df_sim, model, i)

        if resulting_rev > optimal_rate[1]:
            optimal_rate = (new_rate, resulting_rev, original_rate, original_rev)

        else:
            continue
    optimal_prices.append(optimal_rate)
assert len(optimal_prices) == 31, AssertionError("Something went wrong.")

In [17]:
new_rates = []
resulting_revs = []
original_rates = []
original_revs = []
for new_rate, resulting_rev, original_rate, original_rev in optimal_prices:
    new_rates.append(new_rate)
    resulting_revs.append(resulting_rev)
    original_rates.append(original_rate)
    original_revs.append(original_rev)


df_pricing["OptimalRate"] = new_rates    
df_pricing["TRN_RevPickupAtOptimal"] = resulting_revs    
df_pricing["SellingRate"] = original_rates    
df_pricing["TRN_ActualRevPickup"] = original_revs

df_pricing.head()

,id,DOW,RoomsOTB,RevOTB,CxlForecast,TRN_RoomsOTB,TRN_RevOTB,TRN_CxlForecast,TRNP_RoomsOTB,GRP_RoomsOTB,CNT_RoomsOTB,StayDate,STLY_StayDate,DaysUntilArrival,Realized_Cxls,SellingPrice,TM30_RoomsOTB,TM30_RevOTB,TM30_TRN_RoomsOTB,TM30_TRN_RevOTB,TM15_RoomsOTB,TM15_RevOTB,TM15_TRN_RoomsOTB,TM15_TRN_RevOTB,TM05_RoomsOTB,TM05_RevOTB,TM05_TRN_RoomsOTB,TM05_TRN_RevOTB,AsOfDate,STLY_AsOfDate,RemSupply,ADR_OTB,TRN_ADR_OTB,LYA_RoomsSold,LYA_ADR,LYA_RoomRev,LYA_NumCancels,LYA_TRN_RoomsSold,LYA_TRN_ADR,LYA_TRN_RoomRev,ACTUAL_RoomsSold,ACTUAL_ADR,ACTUAL_RoomRev,ACTUAL_TRN_RoomsSold,ACTUAL_TRN_ADR,ACTUAL_TRN_RoomRev,ACTUAL_NumCancels,ACTUAL_RoomsPickup,ACTUAL_ADR_Pickup,ACTUAL_RevPickup,ACTUAL_TRN_RoomsPickup,ACTUAL_TRN_ADR_Pickup,ACTUAL_TRN_RevPickup,MonthNum,DayOfWeek,Mon,Sat,Sun,Thu,Tue,Wed,WE,week_of_year,TM30_ADR_OTB,TM30_RoomsPickup,TM30_RevPickup,TM30_ADR_Pickup,TM30_TRN_ADR_OTB,TM30_TRN_RoomsPickup,TM30_TRN_RevPickup,TM30_TRN_ADR_Pickup,TM15_ADR_OTB,TM15_RoomsPickup,TM15_RevPickup,TM15_ADR_Pickup,TM15_TRN_ADR_OTB,TM15_TRN_RoomsPickup,TM15_TRN_RevPickup,TM15_TRN_ADR_Pickup,TM05_ADR_OTB,TM05_RoomsPickup,TM05_RevPickup,TM05_ADR_Pickup,TM05_TRN_ADR_OTB,TM05_TRN_RoomsPickup,TM05_TRN_RevPickup,TM05_TRN_ADR_Pickup,OTB_GapToLYA_RoomsSold,OTB_GapToLYA_ADR,OTB_GapToLYA_RoomRev,OTB_GapToLYA_NumCancels,OTB_GapToLYA_TRN_RoomsSold,OTB_GapToLYA_TRN_ADR,OTB_GapToLYA_TRN_RoomRev,id_STLY,AsOfDate_STLY,StayDate_STLY,RoomsOTB_STLY,ADR_OTB_STLY,RevOTB_STLY,ACTUAL_RoomsPickup_STLY,ACTUAL_ADR_Pickup_STLY,ACTUAL_RevPickup_STLY,CxlForecast_STLY,RemSupply_STLY,SellingPrice_STLY,Realized_Cxls_STLY,TRN_RoomsOTB_STLY,TRN_ADR_OTB_STLY,TRN_RevOTB_STLY,TRN_CxlForecast_STLY,ACTUAL_TRN_RoomsPickup_STLY,ACTUAL_TRN_ADR_Pickup_STLY,ACTUAL_TRN_RevPickup_STLY,TM30_RoomsPickup_STLY,TM30_ADR_Pickup_STLY,TM30_RevPickup_STLY,TM30_TRN_RoomsPickup_STLY,TM30_TRN_ADR_Pickup_STLY,TM30_TRN_RevPickup_STLY,TM15_RoomsPickup_STLY,TM15_ADR_Pickup_STLY,TM15_RevPickup_STLY,TM15_TRN_RoomsPickup_STLY,TM15_TRN_ADR_Pickup_STLY,TM15_TRN_RevPickup_STLY,TM05_RoomsPickup_STLY,TM05_ADR_Pickup_STLY,TM05_RevPickup_STLY,TM05_TRN_RoomsPickup_STLY,TM05_TRN_ADR_Pickup_STLY,TM05_TRN_RevPickup_STLY,Pace_RoomsOTB,Pace_ADR_OTB,Pace_RevOTB,Pace_CxlForecast,Pace_RemSupply,Pace_SellingPrice,Pace_TRN_RoomsOTB,Pace_TRN_ADR_OTB,Pace_TRN_RevOTB,Pace_TRN_CxlForecast,Pace_TM30_RoomsPickup,Pace_TM30_ADR_Pickup,Pace_TM30_RevPickup,Pace_TM30_TRN_RoomsPickup,Pace_TM30_TRN_ADR_Pickup,Pace_TM30_TRN_RevPickup,Pace_TM15_RoomsPickup,Pace_TM15_ADR_Pickup,Pace_TM15_RevPickup,Pace_TM15_TRN_RoomsPickup,Pace_TM15_TRN_ADR_Pickup,Pace_TM15_TRN_RevPickup,Pace_TM05_RoomsPickup,Pace_TM05_ADR_Pickup,Pace_TM05_RevPickup,Pace_TM05_TRN_RoomsPickup,Pace_TM05_TRN_ADR_Pickup,Pace_TM05_TRN_RevPickup,new_rates,resulting_revs,original_rates,original_revs,OptimalRate,TRN_RevPickupAtOptimal,SellingRate,TRN_ActualRevPickup
11712,2017-08-01 - 2017-08-01,Tue,184.0,35239.14,18.0,151.0,29935.72,16.0,13.0,2.0,18.0,2017-08-01,2016-08-02,0.0,134.0,204.00,170.0,31991.18,139.0,27227.18,180.0,34519.71,147.0,29216.29,177.0,33992.74,144.0,28689.32,2017-08-01,2016-08-02,21.0,191.52,198.25,183.0,170.00,31110.89,81.0,159.0,175.03,27829.05,184.0,191.52,35239.14,151.0,198.25,29935.72,134.0,0.0,0.00,0.00,-1.0,0.00,0.00,8,Tue,False,False,0,False,True,False,False,31.0,188.18,14.0,3247.96,3.34,195.88,12.0,2708.54,2.37,191.78,4.0,719.43,-0.26,198.75,4.0,719.43,-0.50,192.05,7.0,1246.40,-0.53,199.23,7.0,1246.40,-0.98,-1.0,-21.52,-4128.25,-53.0,8.0,-23.22,-2106.67,2016-08-02 - 2016-08-02,2016-08-02,2016-08-02,183.0,170.00,31110.89,0.0,0.00,0.00,39.0,43.0,178.09,81.0,159.0,175.03,27829.05,34.0,0.0,0.00,0.00,3.0,4.00,1230.63,3.0,4.53,1230.63,3.0,2.64,985.79,3.0,2.96,985.79,1.0,1.88,512.25,1.0,2.14,512.25,1.0,21.52,4128.25,-21.0,-22.0,25.91,-8.0,23.22,2106.67,-18.0,11.0,-0.66,2017.33,9.0,-2.16,1477.91,1.0,-2.90,-266.36,1.0,-3.46,-266.36,6.0,-2.41,734.15,6.0,-3.12,734.15,153.00,-196.860000,204.00,-263.840000,153.00,-196.860000,204.00,-263.840000
11713,2017-08-01 - 2017-08-02,

In [18]:
df_pricing.shape

(31, 168)

In [9]:
pd.DataFrame([[0,1,2,3],[4,7,8,9]], columns=['a', 'b', 'c', 'd'])

,a,b,c,d
0,0,1,2,3
1,4,7,8,9


In [10]:
optimal_prices

[(153.0, -196.85999999999999, 204.0, -263.84),
 (155.06, -397.98733333333337, 206.75, -532.0366666666666),
 (157.75, -14.723333333333334, 210.33, -19.6308),
 (263.82, 144.2216, 211.06, 115.37946666666666),
 (264.83, 1135.2379333333333, 211.86, 908.1732000000002),
 (271.36, 1492.48, 217.09, 1193.9950000000001),
 (273.85, 679.148, 219.08, 543.3184),
 (272.41, 526.6593333333334, 217.93, 421.33133333333336),
 (275.23, 488.07453333333336, 220.18, 390.45253333333335),
 (273.19, 1568.1106, 218.55, 1254.477),
 (162.41, -84.4532, 216.55, -112.60600000000001),
 (268.64, 417.2874666666666, 214.91, 333.82686666666666),
 (265.73, 754.6732000000001, 212.58, 603.7272),
 (161.48, -21.530666666666665, 215.31, -25.8372),
 (262.5, 609.0, 210.0, 487.2),
 (160.12, -95.00453333333334, 213.5, -126.67666666666668),
 (270.54, 1432.0584000000001, 216.43, 1145.6361333333332),
 (275.15, 1493.1473333333333, 220.12, 1194.5178666666668),
 (274.74, 1503.7436, 219.79, 1202.9839333333334),
 (270.56, 1468.2389333333333,

In [12]:
optimal_rev = 0
for _, rev, _, _ in optimal_prices:
    optimal_rev += rev
optimal_rev

41379.71473333334

In [13]:
df_pricing.ACTUAL_TRN_RevPickup.sum()

24663.52

In [14]:
24/41

0.5853658536585366

In [ ]:
update_rates(h1_stats, '2017-08-01')